## File transmit through a relay server

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os

def run_transfer(b):
    source_path = source_path_input.value.strip()
    jump_host = jump_host_input.value.strip()
    jump_port = jump_port_input.value
    # 判断传输的是文件还是文件夹，选择不同的命令
    if os.path.isdir(source_path):
        cmd = f"tar cf - {source_path} | ssh -p {jump_port} root@{jump_host} 'nc localhost 3456'"
    else:
        cmd = f"cat {source_path} | ssh -p {jump_port} root@{jump_host} 'nc localhost 3456'"
    
    output_area.clear_output()
    with output_area:
        print("Executing command:")
        print(cmd)
        try:
            result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("Command finished successfully.")
            if result.stdout:
                print("Output:", result.stdout)
        except subprocess.CalledProcessError as e:
            print("Command failed with error:")
            print(e.stderr)

# 定义UI控件
source_path_input = widgets.Text(
    value='',
    placeholder='请输入文件或文件夹路径',
    description='Source:',
    layout=widgets.Layout(width='80%')
)

jump_host_input = widgets.Text(
    value='45.145.74.109',
    placeholder='跳板机IP',
    description='Jump host:',
    layout=widgets.Layout(width='50%')
)

jump_port_input = widgets.IntText(
    value=5222,
    description='SSH Port:',
    layout=widgets.Layout(width='50%')
)

transfer_button = widgets.Button(
    description='开始传输',
    button_style='success'
)

transfer_button.on_click(run_transfer)

output_area = widgets.Output()

# 显示所有控件
display(source_path_input, jump_host_input, jump_port_input, transfer_button, output_area)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import subprocess
from threading import Thread
import time

# 公共参数组件
ssh_params = widgets.VBox([
    widgets.Text(description="跳板机地址:", value='45.145.74.109'),
    widgets.IntText(description="SSH端口:", value=5222),
    widgets.Text(description="用户名:", value='root'),
    widgets.IntText(description="隧道端口:", value=3456)
])

# 模式选择
mode_selector = widgets.RadioButtons(
    options=['发送端', '接收端'],
    description='模式:',
    disabled=False
)

# 文件选择组件
file_selector = widgets.FileUpload(description="选择文件", multiple=True)
folder_selector = widgets.Text(description="文件夹路径:")

# 接收端路径
receiver_path = widgets.Text(description="保存路径:", value='/path/to/received_file')

# 执行按钮
run_btn = widgets.Button(description="执行操作")
output = widgets.Output()

def on_run_clicked(b):
    with output:
        clear_output()
        params = {
            'ssh_host': ssh_params.children[0].value,
            'ssh_port': ssh_params.children[1].value,
            'user': ssh_params.children[2].value,
            'tunnel_port': ssh_params.children[3].value,
            'mode': mode_selector.value
        }
        
        if params['mode'] == '发送端':
            if len(file_selector.value) > 0:
                # 处理上传文件
                for name, content in file_selector.value.items():
                    with open(name, 'wb') as f:
                        f.write(content['content'])
                src_path = ' '.join(file_selector.value.keys())
            else:
                src_path = folder_selector.value
                
            cmd = f"tar cf - {src_path} | ssh -p {params['ssh_port']} {params['user']}@{params['ssh_host']} 'nc localhost {params['tunnel_port']}'"
            
        else:
            cmd = f"nc -l {params['tunnel_port']} | tar xf -"
            if receiver_path.value.strip():
                cmd = f"nc -l {params['tunnel_port']} > {receiver_path.value}"
            
            # 在后台启动隧道
            tunnel_cmd = f"ssh -N -R {params['tunnel_port']}:localhost:{params['tunnel_port']} -p {params['ssh_port']} {params['user']}@{params['ssh_host']}"
            Thread(target=lambda: subprocess.run(tunnel_cmd, shell=True)).start()
            time.sleep(1)  # 等待隧道建立

        print("执行命令:", cmd)
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        print(result.stdout)
        if result.stderr:
            print("错误信息:", result.stderr)

run_btn.on_click(on_run_clicked)

# 动态显示组件
def update_ui(change):
    if change['new'] == '发送端':
        special_params.children = [file_selector, folder_selector]
    else:
        special_params.children = [receiver_path]

mode_selector.observe(update_ui, names='value')

special_params = widgets.VBox()
ui = widgets.VBox([
    mode_selector,
    ssh_params,
    special_params,
    run_btn,
    output
])

# 初始UI状态
update_ui({'new': mode_selector.value})
display(ui)

In [ ]:
# -*- coding: utf-8 -*-
"""
安全文件传输系统（通过跳板机）
使用说明：
1. 运行所有代码单元格
2. 在最后的小部件界面填写参数
3. 点击"执行传输"按钮开始操作
"""
# 1. 依赖安装（首次使用需要运行）
# !pip install paramiko scp tqdm ipywidgets --quiet

# 2. 核心库导入
import os
import paramiko
from scp import SCPClient, SCPException
from tqdm.auto import tqdm
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display, clear_output
import traceback

# 3. SCP进度显示类
class ProgressBar(tqdm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._current = 0

    def update_to(self, transferred, total):
        if self.total is None and total:
            self.reset(total=total)
        self.update(transferred - self._current)
        self._current = transferred

# 4. 文件传输核心类
class SecureFileTransfer:
    def __init__(self):
        self.ssh = None
        self.scp = None
        
    def _connect(self, host, port, user, password):
        """建立SSH连接"""
        try:
            self.ssh = paramiko.SSHClient()
            self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            self.ssh.connect(
                hostname=host,
                port=int(port),
                username=user,
                password=password,
                timeout=10
            )
            return True
        except Exception as e:
            self._show_error(f"连接失败: {str(e)}")
            return False

    def _transfer_file(self, src, dst, operation):
        """执行文件传输"""
        try:
            with SCPClient(self.ssh.get_transport(), progress=self._progress_handler) as scp:
                if operation == 'send':
                    if os.path.isdir(src):
                        scp.put(src, remote_path=dst, recursive=True)
                    else:
                        scp.put(src, remote_path=dst)
                else:
                    scp.get(remote_path=src, local_path=dst, recursive=True)
            return True
        except (SCPException, FileNotFoundError) as e:
            self._show_error(f"传输错误: {str(e)}")
            return False

    def _progress_handler(self, filename, size, transferred):
        """处理进度显示"""
        self.progress_bar.total = size
        self.progress_bar.set_description(f"传输 {filename}")
        self.progress_bar.update_to(transferred, size)

    def _show_error(self, msg):
        """显示错误信息"""
        with self.output:
            clear_output()
            print(f"❌ {msg}")

    def run_transfer(self, params):
        """执行传输主逻辑"""
        self.output = widgets.Output()
        display(self.output)
        
        with self.output:
            # 验证参数有效性
            if not all([params['host'], params['user'], params['password']]):
                self._show_error("请填写所有必填参数")
                return

            # 建立连接
            print("🔄 正在连接跳板机...")
            if not self._connect(params['host'], params['port'], 
                              params['user'], params['password']):
                return

            try:
                # 初始化进度条
                self.progress_bar = ProgressBar(
                    unit='B', unit_scale=True, mininterval=0.5
                )

                # 执行传输
                print("🚀 开始传输...")
                success = self._transfer_file(
                    src=params['local_path'],
                    dst=params['remote_path'],
                    operation=params['operation']
                )

                if success:
                    print("✅ 传输成功完成！")
                    self.progress_bar.close()
                else:
                    self._show_error("传输未完成")

            except Exception as e:
                self._show_error(f"意外错误: {traceback.format_exc()}")
            finally:
                if self.ssh:
                    self.ssh.close()

# 5. 用户界面组件
class TransferUI:
    def __init__(self):
        # 输入组件
        self.host = widgets.Text(description="跳板机IP:", placeholder='192.168.1.100')
        self.port = widgets.IntText(description="SSH端口:", value=22)
        self.user = widgets.Text(description="用户名:", placeholder='admin')
        self.password = widgets.Password(description="密码:")
        
        # 路径组件
        self.local_path = widgets.Text(
            description="本地路径:", 
            placeholder='/path/to/local/file_or_folder'
        )
        self.remote_path = widgets.Text(
            description="远程路径:", 
            placeholder='~/destination/path'
        )
        
        # 操作类型
        self.operation = widgets.RadioButtons(
            options=['send', 'receive'],
            description='操作类型:',
            disabled=False
        )
        
        # 执行按钮
        self.execute_btn = widgets.Button(
            description='执行传输',
            button_style='success',
            icon='rocket'
        )
        self.execute_btn.on_click(self._execute)

        # 组装界面
        self.ui = VBox([
            HBox([self.host, self.port]),
            HBox([self.user, self.password]),
            self.local_path,
            self.remote_path,
            self.operation,
            self.execute_btn
        ], layout=Layout(padding='20px'))

    def _execute(self, btn):
        """收集参数并启动传输"""
        params = {
            'host': self.host.value,
            'port': self.port.value,
            'user': self.user.value,
            'password': self.password.value,
            'local_path': self.local_path.value,
            'remote_path': self.remote_path.value,
            'operation': self.operation.value
        }
        transfer = SecureFileTransfer()
        transfer.run_transfer(params)

    def show(self):
        display(self.ui)

# 6. 启动界面
TransferUI().show()

# SFTP Transfer Tool - Usage Guide

This document explains how to use the SFTP-based transfer tool in the `file_transmission_b.ipynb` notebook.

## How It Works

Unlike the other transfer methods that require coordination between sender and receiver, the SFTP method handles the complete transfer process in a single operation:

1. The tool establishes an SSH connection to the relay server
2. It uses the SFTP protocol (built on SSH) to transfer files securely
3. Progress is tracked with real-time updates

## Usage Instructions

### For Sending Files TO Remote Server:

1. Fill in the connection details:
   - Jump server IP address
   - SSH port (usually 22)
   - Username and password

2. Set the paths:
   - **Local path**: The file or folder on your machine you want to send
   - **Remote path**: The destination path on the remote server

3. Select operation type: **send**

4. Click "执行传输" to start the transfer

### For Receiving Files FROM Remote Server:

1. Fill in the same connection details as above

2. Set the paths:
   - **Local path**: Where you want to save the file/folder on your machine
   - **Remote path**: The file or folder on the remote server you want to download

3. Select operation type: **receive**

4. Click "执行传输" to start the transfer

## Key Features

- **Resume Support**: If a transfer is interrupted, it can be resumed from where it left off
- **Progress Tracking**: Real-time progress bars show transfer speed and completion percentage
- **Directory Support**: Can transfer entire directories with nested folder structures
- **Secure Transfer**: All data is encrypted during transmission

## Troubleshooting

- **Connection Issues**: Verify the server IP, port, and credentials are correct
- **Permission Errors**: Ensure you have read/write permissions for the specified paths
- **Path Format**: Use absolute paths for best results (e.g., `/home/user/files` rather than `~/files`)

## Advantages Over Basic Transfer Methods

This SFTP-based approach is superior to the netcat methods because:
1. No coordination needed between sender and receiver
2. Built-in encryption for secure transfers
3. Resume capability for interrupted transfers
4. Automatic directory structure preservation


In [1]:
# -*- coding: utf-8 -*-
"""
安全文件传输系统（通过跳板机，使用SFTP协议）
使用说明：
1. 运行所有代码单元格
2. 在最后的小部件界面填写参数
3. 点击"执行传输"按钮开始操作

特点:
- 使用SFTP协议（比SCP更安全、更强大）
- 支持断点续传功能
- 更加鲁棒的错误处理
"""
# 1. 依赖安装（首次使用需要运行）
# !pip install paramiko tqdm ipywidgets --quiet

from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display, clear_output
from Secure_file_transfer import SecureFileTransfer

# 5. 用户界面组件
class TransferUI:
    def __init__(self):
        # 输入组件
        self.jump_server = widgets.Text(description="跳板机IP:", placeholder='192.168.1.100')
        self.port = widgets.IntText(description="SSH端口:", value=22)
        self.user = widgets.Text(description="用户名:", placeholder='admin')
        self.password = widgets.Password(description="密码:")
        
        # 路径组件
        self.local_path = widgets.Text(
            description="本地路径:", 
            placeholder='/path/to/local/file_or_folder'
        )
        self.remote_path = widgets.Text(
            description="远程路径:", 
            placeholder='~/destination/path'
        )
        
        # 操作类型
        self.operation = widgets.RadioButtons(
            options=['send', 'receive'],
            description='操作类型:',
            disabled=False
        )
        
        # 执行按钮
        self.execute_btn = widgets.Button(
            description='执行传输',
            button_style='success',
            icon='rocket'
        )
        self.execute_btn.on_click(self._execute)

        # 状态提示
        self.status_text = widgets.HTML(
            value='<div style="color:#666;font-style:italic;">准备就绪，请填写参数</div>'
        )
        
        # 组装界面
        self.ui = VBox([
            HBox([self.jump_server, self.port]),
            HBox([self.user, self.password]),
            self.local_path,
            self.remote_path,
            self.operation,
            HBox([self.execute_btn, self.status_text], layout=Layout(align_items='center')),
            widgets.HTML(value="""
            <div style="background:#e9f7f0;padding:10px;border-radius:5px;margin-top:10px;">
                <h4 style="margin-top:0;color:#2e7d32;">📝 使用说明</h4>
                <ul style="padding-left:20px;margin-bottom:0;">
                    <li>本工具支持<b>文件</b>和<b>文件夹</b>的上传和下载</li>
                    <li>支持<b>断点续传</b>功能，传输中断后可从断点处继续</li>
                    <li>如遇到传输问题，可查看输出区域的错误信息</li>
                </ul>
            </div>
            """)
        ], layout=Layout(padding='20px'))

    def _execute(self, btn):
        """收集参数并启动传输"""
        # 更新状态
        self.status_text.value = '<div style="color:#1976d2;font-weight:bold;">🔄 任务处理中...</div>'
        
        params = {
            'jump_server': self.jump_server.value,
            'port': self.port.value,
            'user': self.user.value,
            'password': self.password.value,
            'local_path': self.local_path.value,
            'remote_path': self.remote_path.value,
            'operation': self.operation.value
        }
        
        transfer = SecureFileTransfer()
        
        # 禁用按钮，防止重复点击
        self.execute_btn.disabled = True
        
        try:
            transfer.run_transfer(params)
            self.status_text.value = '<div style="color:#388e3c;font-weight:bold;">✅ 任务完成</div>'
        except Exception:
            self.status_text.value = '<div style="color:#d32f2f;font-weight:bold;">❌ 任务失败</div>'
        finally:
            # 重新启用按钮
            self.execute_btn.disabled = False

    def show(self):
        display(self.ui)

# 6. 启动界面
TransferUI().show()

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from jump_server_transfer import JumpServerTransfer
import threading

# 创建交互式组件
hostname_input = widgets.Text(
    description="跳板机地址:",
    value="20.30.80.200",
    placeholder="20.30.80.200"
)
port_input = widgets.IntText(description="端口:", value=22)
username_input = widgets.Text(
    description="用户名:",
    value="test_wj",
    placeholder="test_wj"
)
password_input = widgets.Password(description="密码:")
local_path_input = widgets.Text(
    description="本地路径:",
    placeholder="/path/to/local/file_or_folder"
)
remote_path_input = widgets.Text(
    description="跳板机缓存路径(可选):",
    value="~/file_transfer_cache",
    placeholder="~/file_transfer_cache"
)
operation_type_input = widgets.Dropdown(description="操作:", options=["发送方", "接收方"])
start_button = widgets.Button(description="开始传输")
output = widgets.Output()

# 创建全局变量，保存发送端和接收端的ssh对象
sender_ssh = None  
receiver_ssh = None

# 定义开始传输的回调函数
def on_start_button_clicked(b):
    with output:
        clear_output()  # 清除之前的输出
        try:
            # 获取用户输入
            hostname = hostname_input.value
            port = port_input.value
            username = username_input.value
            password = password_input.value
            local_path = local_path_input.value
            remote_cache_path = remote_path_input.value
            operation_type = operation_type_input.value
            
            # 先创建或获取SSH实例
            ssh_instance = create_ssh_instance(operation_type)

            if operation_type == "发送方":
                # 使用线程执行发送操作
                def send_thread():
                    ssh_instance.transfer(local_path, operation_type, receiver_ssh.ssh if receiver_ssh else None)
                sender_thread = threading.Thread(target=send_thread)
                sender_thread.start()

            elif operation_type == "接收方":
                 # 使用线程执行接收操作
                def receive_thread():
                    ssh_instance.transfer(local_path, operation_type, sender_ssh.ssh if sender_ssh else None)
                receiver_thread = threading.Thread(target=receive_thread)
                receiver_thread.start()

        except Exception as e:
            print(f"发生错误: {e}")

def create_ssh_instance(operation_type):
    global sender_ssh, receiver_ssh
    hostname = hostname_input.value
    port = port_input.value
    username = username_input.value
    password = password_input.value
    remote_cache_path = remote_path_input.value

    if operation_type == "发送方":
        if sender_ssh is None:  # 避免重复创建
            sender_ssh = JumpServerTransfer(hostname, port, username, password, remote_cache_path)
            sender_ssh.connect()
        return sender_ssh
    else:  # 接收方
        if receiver_ssh is None: # 避免重复创建
            receiver_ssh = JumpServerTransfer(hostname, port, username, password, remote_cache_path)
            receiver_ssh.connect()
        return receiver_ssh

# 绑定回调函数 - 只绑定一次
start_button.on_click(on_start_button_clicked)

# 显示界面
display(hostname_input, port_input, username_input, password_input,
        local_path_input, remote_path_input, operation_type_input, start_button, output)


Text(value='20.30.80.200', description='跳板机地址:', placeholder='20.30.80.200')

IntText(value=22, description='端口:')

Text(value='test_wj', description='用户名:', placeholder='test_wj')

Password(description='密码:')

Text(value='', description='本地路径:', placeholder='/path/to/local/file_or_folder')

Text(value='~/file_transfer_cache', description='跳板机缓存路径(可选):', placeholder='~/file_transfer_cache')

Dropdown(description='操作:', options=('发送方', '接收方'), value='发送方')

Button(description='开始传输', style=ButtonStyle())

Output()